# Description

Notebook for custom sklearn wrappers for exclusion threshold classifier.

# Imports

In [5]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import sys

from ipywidgets import interact_manual
from scipy.stats import norm
from scipy.special import logit
from sklearn.base import clone
from sklearn.datasets import make_regression
from sklearn.model_selection import KFold, GridSearchCV
from linearmodels.iv import IV2SLS

# Common issue with glibc, scikit (?), where the number of libraries loaded with static TLS is limited
# Can only load one of econml or rpy2...
# e.g. https://github.com/pytorch/pytorch/issues/2575#issuecomment-523657178
from econml.dml import CausalForestDML, LinearDML
from econml.grf import CausalForest

# user imports
sys.path.append("../")

# Don't import iv_power for now since it needs rpy2
from utils.pwr import rdd_power
from utils.sim import generate_IV_comply_indicator

# Sandbox wrappers

- executive decision: probably easier to just write a custom fold validation
    - problem with using sklearn gridsearchCV: would be retraining the model on the same data for each threshold tested -> slowwww

In [218]:
def iv_neff(scores, tz_df, threshold):
    """
    Computes the effective sample size of the given data sample.
    
    Args:
        scores (np.array): array of compliance scores
        tz_df (pd.DataFrame)
    
    """    
    tz_df['score'] = scores
    sel_df = tz_df[tz_df['score'] >= threshold].copy()
    
    if (sel_df[(sel_df['Z'] == 1)].shape[0] == 0) or (sel_df[(sel_df['Z'] == 0)].shape[0] == 0):
        #print("no samples!")
        return -np.inf
    
    comply_rate = sel_df[(sel_df['Z'] == 1)]['T'].mean() - sel_df[(sel_df['Z'] == 0)]['T'].mean()
    
    return sel_df.shape[0] * (comply_rate**2)

In [219]:
class ThresholdCV():
    """
    Custom CV operation to detect the best threshold.
    
    Could also be converted to an estimator to be
    nested with GridSearchCV, down the line.
    """
    default_n_thresholds = 100
    random_state = 42
    
    def __init__(self, thresholds=None, n_splits=5, clf_class=None):
        self.thresholds = thresholds
        self.n_splits = n_splits
        
        if clf_class:
            self.clf_class = clf_class
        else:
            self.clf_class = CausalForest
        
    def get_best_threshold(self, iv_df, feat_cols):
        """
        Returns the best threshold based on KFold fitting.
        
        Args:
            iv_df (pd.DataFrame): the IV data
            feat_cols (list): the list of column name features
        """
        kfold = KFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)
        
        # each row is the neff score for all thresholds on a particular fold
        # each column holds all the fold scores for a particular threshold
        if self.thresholds is not None:
            neff_scores = np.zeros((self.n_splits, len(self.thresholds)))
        else:
            neff_scores = np.zeros((self.n_splits, self.default_n_thresholds))
        
        for split_idx, (train_idx, test_idx) in enumerate(kfold.split(iv_df)):
            # print(idx)
            # print(train_idx)
            # print(test_idx)
            train_df = iv_df.iloc[train_idx].copy()
            test_df = iv_df.iloc[test_idx].copy()
            
            # train a single classifier for a given split
            train_X = train_df[feat_cols]
            train_Y = train_df['T']
            train_T = train_df['Z']
            
            test_X = test_df[feat_cols]
            test_Y = test_df['T']
            test_T = test_df['Z']
            
            # initialize the class constructor
            clf = self.clf_class(random_state=self.random_state)
            clf.fit(X=train_X, y=train_Y, T=train_T)
            #print(clf)
            
            scores = clf.predict(test_X)
            tz_df = test_df[['T', 'Z']].copy()
            
            # go over all thresholds with fixed clf
            if self.thresholds is None:
                self.thresholds = np.linspace(np.min(scores), np.max(scores), 
                                              num=self.default_n_thresholds).flatten()
            
            #print(self.thresholds)
            
            for thres_idx, threshold in enumerate(self.thresholds):
                neff = iv_neff(scores, tz_df, threshold)
                neff_scores[split_idx, thres_idx] = neff
    
        mean_neff = np.mean(neff_scores, axis=0)
        best_threshold = self.thresholds[np.argmax(mean_neff)]
        
        assert mean_neff.shape[0] == len(thresholds)
        
        print("Best neff: {:.3f}".format(np.max(mean_neff)))
        print("Best threshold: {:.3f}".format(best_threshold))
        return best_threshold
                
    

## Make data: strong compliance

In [247]:
seed = 42
tau = 0.25
n_samples = 2000
n_feats = 5
regression_dict = dict(n_informative=5, noise=0, n_features=n_feats)

iv_df = generate_IV_comply_indicator(n_samples=n_samples, 
                                    tau=tau,
                                    prop_nt=0.30,
                                    prop_at=0.30,
                                    use_covars=True,
                                    regression_dict=regression_dict,
                                    seed=seed)

feat_cols = ['feat_{}'.format(i) for i in range(n_feats)]

In [248]:
n_splits = 2

kfold = KFold(n_splits=n_splits,
             shuffle=True, random_state=42)

indices = []

# TODO be careful on what to split on
for train_idx, test_idx in kfold.split(iv_df): #, y=iv_df['Z']
    indices.append(test_idx)
    

s1_df = iv_df.iloc[indices[0]].copy()
s2_df = iv_df.iloc[indices[1]].copy()

In [249]:
%%time

thresholds = np.linspace(0, 1, num=100)
threshold_clf = ThresholdCV()#thresholds=thresholds)
best_threshold = threshold_clf.get_best_threshold(s1_df, feat_cols)

Best neff: 68.745
Best threshold: 0.466
CPU times: user 4.54 s, sys: 499 ms, total: 5.04 s
Wall time: 3.77 s


In [253]:
# sanity check: no exclusion
print("no exclusion, s2_df neff", iv_neff([1] * int(n_samples / 2), s2_df, 0))
#print(iv_neff([1] * n_samples, iv_df, 0))

no exclusion, s2_df neff 165.21032258675464


In [254]:
# sanity check, fit model
s1_X = s1_df[feat_cols]
s1_Y = s1_df['T'].values
s1_T = s1_df['Z'].values

s2_X = s2_df[feat_cols]
s2_Y = s2_df['T'].values
s2_T = s2_df['Z'].values

clf = CausalForest()
clf.fit(X=s1_X, y=s1_Y, T=s1_T)

s2_scores = clf.predict(s2_X)

iv_neff(s2_scores, s2_df, best_threshold)

## Make data: weak compliance

In [270]:
seed = 42
tau = 0.25
n_samples = 2000
n_feats = 5
regression_dict = dict(n_informative=5, noise=50, n_features=n_feats)

iv_df = generate_IV_comply_indicator(n_samples=n_samples, 
                                    tau=tau,
                                    prop_nt=0.30,
                                    prop_at=0.30,
                                    use_covars=True,
                                    regression_dict=regression_dict,
                                    seed=seed)

feat_cols = ['feat_{}'.format(i) for i in range(n_feats)]

In [271]:
n_splits = 2

kfold = KFold(n_splits=n_splits,
             shuffle=True, random_state=42)

indices = []

# TODO be careful on what to split on
for train_idx, test_idx in kfold.split(iv_df): #, y=iv_df['Z']
    indices.append(test_idx)
    

s1_df = iv_df.iloc[indices[0]].copy()
s2_df = iv_df.iloc[indices[1]].copy()

In [272]:
%%time

thresholds = np.linspace(0, 1, num=100)
threshold_clf = ThresholdCV()#thresholds=thresholds)
best_threshold = threshold_clf.get_best_threshold(s1_df, feat_cols)

Best neff: 48.486
Best threshold: 0.452
CPU times: user 4.41 s, sys: 540 ms, total: 4.95 s
Wall time: 3.56 s


In [273]:
# sanity check: no exclusion
print("no exclusion, s2_df neff", iv_neff([1] * int(n_samples / 2), s2_df, 0))
#print(iv_neff([1] * n_samples, iv_df, 0))

no exclusion, s2_df neff 179.30473957097647


In [274]:
# sanity check, fit model
s1_X = s1_df[feat_cols]
s1_Y = s1_df['T'].values
s1_T = s1_df['Z'].values

s2_X = s2_df[feat_cols]
s2_Y = s2_df['T'].values
s2_T = s2_df['Z'].values

clf = CausalForest()
clf.fit(X=s1_X, y=s1_Y, T=s1_T)

s2_scores = clf.predict(s2_X)

iv_neff(s2_scores, s2_df, best_threshold)

## Make data: noise

In [275]:
seed = 42
tau = 0.25
n_samples = 2000
n_feats = 5
regression_dict = dict(n_informative=0, noise=100, n_features=n_feats)

iv_df = generate_IV_comply_indicator(n_samples=n_samples, 
                                    tau=tau,
                                    prop_nt=0.30,
                                    prop_at=0.30,
                                    use_covars=True,
                                    regression_dict=regression_dict,
                                    seed=seed)

feat_cols = ['feat_{}'.format(i) for i in range(n_feats)]

In [276]:
n_splits = 2

kfold = KFold(n_splits=n_splits,
             shuffle=True, random_state=42)

indices = []

# TODO be careful on what to split on
for train_idx, test_idx in kfold.split(iv_df): #, y=iv_df['Z']
    indices.append(test_idx)
    

s1_df = iv_df.iloc[indices[0]].copy()
s2_df = iv_df.iloc[indices[1]].copy()

In [277]:
%%time

thresholds = np.linspace(0, 1, num=100)
threshold_clf = ThresholdCV()#thresholds=thresholds)
best_threshold = threshold_clf.get_best_threshold(s1_df, feat_cols)

Best neff: 31.776
Best threshold: 0.176
CPU times: user 4.63 s, sys: 448 ms, total: 5.07 s
Wall time: 3.6 s


In [278]:
# sanity check: no exclusion
print("no exclusion, s2_df neff", iv_neff([1] * int(n_samples / 2), s2_df, 0))
#print(iv_neff([1] * n_samples, iv_df, 0))

no exclusion, s2_df neff 174.724


In [279]:
# sanity check, fit model
s1_X = s1_df[feat_cols]
s1_Y = s1_df['T'].values
s1_T = s1_df['Z'].values

s2_X = s2_df[feat_cols]
s2_Y = s2_df['T'].values
s2_T = s2_df['Z'].values

clf = CausalForest()
clf.fit(X=s1_X, y=s1_Y, T=s1_T)

s2_scores = clf.predict(s2_X)

iv_neff(s2_scores, s2_df, best_threshold)

## Custom GridSearchCV sandbox

In [4]:
from sklearn.metrics import make_scorer

In [118]:
def iv_neff_score(y, y_pred, data=None):
    """
    Computes the effective sample size of the given data sample.
    
    Args:
        y (np.array): defined to match the function signature of a score, not used. 
        y_pred (np.array): binary indicator of whether a sample is included (1) or excluded (0)
        data (pd.DataFrame): the data frame containing T and Z
    
    """    
    data['sel'] = y_pred
    #print(data)
    sel_df = data[data['sel'] == 1].copy()
    
    comply_rate = sel_df[(sel_df['Z'] == 1)]['T'].mean() - sel_df[(sel_df['Z'] == 0)]['T'].mean()
    
    return sel_df.shape[0] * (comply_rate**2)

In [120]:
neff_scorer = make_scorer(iv_neff_score)

In [89]:
class ExcludeClassifier(CausalForest):
    """
    Wrapper class for CausalForest, with a threshold.
    
    TODO modify so that y actually contains both Z and T, which can then be passed into scorer as "y"
    "y_pred" should still be allowed as a single score.
    """
    def __init__(self, threshold=0.5, n_estimators=100):
        self.threshold = threshold
        super().__init__(n_estimators=n_estimators)
    
    def fit(self, X, y):
        """
        Here, to hack around the sklearn API, y is actually a df with 'T' and 'Z' values.
        """
        super().fit(X, T, y)
    
    def predict(self, X):
        preds = super().predict(X)
        
        return preds > self.threshold

In [53]:
n_splits = 2

kfold = KFold(n_splits=n_splits,
             shuffle=True, random_state=42)

indices = []

# TODO be careful on what to split on
for train_idx, test_idx in kfold.split(iv_df): #, y=iv_df['Z']
    indices.append(test_idx)
    

s1_df = iv_df.iloc[indices[0]].copy()
s2_df = iv_df.iloc[indices[1]].copy()

In [54]:

X = s1_df[feat_cols]
Y = s1_df['T'].values
T = s1_df['Z'].values

In [95]:
cforest = ExcludeClassifier()
cforest_cv = GridSearchCV(estimator=cforest,
                          param_grid={
                              "n_estimators": [100],
                              "threshold": [0.5, 0.7, 0.8]
                          },
                          scoring="accuracy", 
                          verbose=4)

In [96]:
%%time
cforest_cv.fit(X=X, y=Y, T=T)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END ...n_estimators=100, threshold=0.5;, score=0.570 total time=   0.4s
[CV 2/5] END ...n_estimators=100, threshold=0.5;, score=0.555 total time=   0.3s
[CV 3/5] END ...n_estimators=100, threshold=0.5;, score=0.565 total time=   0.3s
[CV 4/5] END ...n_estimators=100, threshold=0.5;, score=0.475 total time=   0.3s
[CV 5/5] END ...n_estimators=100, threshold=0.5;, score=0.570 total time=   0.3s
[CV 1/5] END ...n_estimators=100, threshold=0.7;, score=0.510 total time=   0.3s
[CV 2/5] END ...n_estimators=100, threshold=0.7;, score=0.485 total time=   0.3s
[CV 3/5] END ...n_estimators=100, threshold=0.7;, score=0.555 total time=   0.3s
[CV 4/5] END ...n_estimators=100, threshold=0.7;, score=0.455 total time=   0.3s
[CV 5/5] END ...n_estimators=100, threshold=0.7;, score=0.515 total time=   0.3s
[CV 1/5] END ...n_estimators=100, threshold=0.8;, score=0.500 total time=   0.3s
[CV 2/5] END ...n_estimators=100, threshold=0.8;,

GridSearchCV(estimator=ExcludeClassifier(),
             param_grid={'n_estimators': [100], 'threshold': [0.5, 0.7, 0.8]},
             scoring='accuracy', verbose=4)

In [97]:
cforest_cv.best_params_

{'n_estimators': 100, 'threshold': 0.5}